In [ ]:
from apps.integrations.helpers.maximo import EngieMaximoIntegration
from helpers.signals import DisableSignals
from helpers.strings import to_snake_case
from tqdm.notebook import tqdm

In [ ]:
company = Company.objects.get(name="GSB - UHE-PASSO FUNDO")

In [ ]:
for a in tqdm(
    OccurrenceRecord.objects.filter(company=company)
):
    og_form_data = a.form_data
    new_form_data = {}
    for key, value in og_form_data.items():
        new_form_data[to_snake_case(key)] = value
    a.form_data = new_form_data
    with DisableSignals():
        a.save()

In [ ]:
for integration in IntegrationConfig.objects.filter(
    company=company, 
    instrument_operational_position__isnull=True
):
    integration.instrument_operational_position = str(int(integration.reading_operational_control.kind) - 701)
    print(integration.instrument_operational_position)
    integration.save()

In [ ]:
for integration in IntegrationConfig.objects.filter(company=company):
    integration_class = EngieMaximoIntegration(integration)
    integration_class.run()